In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/sujit25/anaconda3/envs/ml_env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets("../Data/MNIST_data", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../Data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../Data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../Data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# define computation graph

In [3]:
num_epochs= 100
batch_size = 50
learning_rate = 1e-4

In [4]:
def init_weights(t_shape):
    return tf.Variable(dtype=tf.float32, initial_value = tf.random_uniform(t_shape, minval=0, maxval=1, dtype=tf.float32))

In [5]:
def init_bias(t_shape):
    return tf.Variable(initial_value = tf.zeros(shape=t_shape), dtype= tf.float32)

In [6]:
def conv2d(x, W):
    # x -> [num_images, image_height, image_width, num_channels]
    # W -> [filter_h, filter_w, num_channels, num_filters]
    return tf.nn.conv2d(input=x, filter=W, strides=[1,1,1,1], padding='SAME')

In [7]:
def relu_activation(x):
    return tf.nn.relu(x)

In [8]:
def maxpool_2by2(x):
    # x -> [num_images, image_height, image_width, num_channels]
    # ksize = window size,
    # strides =  
    return tf.nn.max_pool(value=x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [9]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    z = tf.add(conv2d(input_x, W), b)
    a = tf.nn.relu(z)
    return a

In [10]:
def fully_connected_layer(prev_activation, input_layer_size, output_layer_size):
    hidden_weights = init_weights([input_layer_size, output_layer_size])
    bias = init_bias([output_layer_size])
    return tf.add(tf.matmul(prev_activation, hidden_weights), bias)

In [11]:
# Placeholders
x = tf.placeholder( dtype= tf.float32, shape = [None, 28*28])
y = tf.placeholder( dtype= tf.float32, shape = [None, 10])

# Layers
# reshape input tensor x 
x_image = tf.reshape(x, shape= [-1,28,28,1])

# conv1 layer
z1 = convolutional_layer(x_image, [5,5,1,32])

# relu layer 1
a1 = relu_activation(z1)

# max pool layer 1
p1 = maxpool_2by2(a1)

# conv2 layer
z2 = convolutional_layer(p1, [5,5,32,64])

# relu 2 layer
a2 = relu_activation(z2)

# max pool layer 2
p2 = maxpool_2by2(a2)

# flatout 
p2_flatten_out = tf.reshape(p2, shape=[-1, 7*7*64])

# fc1
# p2_flatten_out - num_images, 3136
# input_layer_size - 3136
# output_layer_size - 10
z3 = fully_connected_layer(p2_flatten_out,7*7*64,10)

# loss function - compute overall loss
cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits= z3))

# optimizer
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)

# accuracy 
correct_predictions = tf.equal(tf.argmax(z3, axis=1), tf.argmax(y, axis=1)) 
accuracy = tf.reduce_mean(tf.cast(correct_predictions, dtype= tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
# run computational graph

# configuration for allowing usage of gpu with requirement
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.log_device_placement=True

with tf.Session() as sess:
    
    init = tf.global_variables_initializer()
    
    # initialize global variables
    sess.run(init)
    cost_hist= []
    # iterate over epochs
    for epoch_num in range(num_epochs):
        epoch_cost = 0
        
        num_batches = int(mnist_data.train.num_examples/ batch_size)
        
        # iterate over batches
        for batch_index in range(num_batches):
            batch_cost = 0
            batch_x, batch_y = mnist_data.train.next_batch(batch_size=batch_size)
            batch_cost,_ = sess.run([cross_entropy_loss, train], feed_dict = {x: batch_x, y: batch_y})
            epoch_cost += batch_cost
        
        cost_hist.append(epoch_cost)
        
        # compute train accuracy
        train_acc = sess.run(accuracy, feed_dict ={x : mnist_data.train.images, y: mnist_data.train.labels})
        
        # compute test accuracy
        test_acc = sess.run(accuracy, feed_dict ={x : mnist_data.test.images, y:mnist_data.test.labels})
        
        # print cost
        if epoch_num % 100 == 0:
            print("epoch: {}, cost: {}, train acc: {}, test acc: {}".format(epoch_num, epoch_cost, train_acc, test_acc))